In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
# sc = SparkContext(master = 'local[2]')

In [3]:
spark = (SparkSession 
    .builder 
    .master("local[2]")
    .appName("Python Spark SQL basic example") 
    .config("spark.some.config.option", "some-value") 
    .getOrCreate())
sc = spark.sparkContext

In [4]:
rdd = sc.parallelize([('a',7),('a',2),('b',2)])

In [5]:
rdd.collect()

[('a', 7), ('a', 2), ('b', 2)]

In [6]:
from pyspark.ml.fpm import FPGrowth

df = spark.createDataFrame([
    (0, [1, 2, 5]),
    (1, [1, 2, 3, 5]),
    (2, [1, 2])
], ["id", "items"])

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.5, minConfidence=0.6)
model = fpGrowth.fit(df)

# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(df).show()

+---------+----+
|    items|freq|
+---------+----+
|      [5]|   2|
|   [5, 1]|   2|
|[5, 1, 2]|   2|
|   [5, 2]|   2|
|      [2]|   3|
|      [1]|   3|
|   [1, 2]|   3|
+---------+----+

+----------+----------+------------------+----+
|antecedent|consequent|        confidence|lift|
+----------+----------+------------------+----+
|       [5]|       [1]|               1.0| 1.0|
|       [5]|       [2]|               1.0| 1.0|
|    [1, 2]|       [5]|0.6666666666666666| 1.0|
|    [5, 2]|       [1]|               1.0| 1.0|
|    [5, 1]|       [2]|               1.0| 1.0|
|       [2]|       [5]|0.6666666666666666| 1.0|
|       [2]|       [1]|               1.0| 1.0|
|       [1]|       [5]|0.6666666666666666| 1.0|
|       [1]|       [2]|               1.0| 1.0|
+----------+----------+------------------+----+

+---+------------+----------+
| id|       items|prediction|
+---+------------+----------+
|  0|   [1, 2, 5]|        []|
|  1|[1, 2, 3, 5]|        []|
|  2|      [1, 2]|       [5]|
+---+--

In [7]:
from pyspark.ml.feature import Word2Vec

# Input data: Each row is a bag of words from a sentence or document.
documentDF = spark.createDataFrame([
    ("Hi I heard about Spark".split(" "), ),
    ("I wish Java could use case classes".split(" "), ),
    ("Logistic regression models are neat".split(" "), )
], ["text"])

# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(documentDF)

result = model.transform(documentDF)
for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

Text: [Hi, I, heard, about, Spark] => 
Vector: [-0.011087720049545169,-0.04713543206453324,-0.030747681856155396]

Text: [I, wish, Java, could, use, case, classes] => 
Vector: [0.02089989078896386,0.025788720164980204,-0.029980793063129695]

Text: [Logistic, regression, models, are, neat] => 
Vector: [0.03326573260128498,0.010403224825859071,-0.04281422272324562]

